# DINOv2를 사용한 이미지 분류

Meta Research에서 2023년 4월에 출시한 DINOv2는 컴퓨터 비전 모델을 훈련하는 자체 지도(self-supervised method) 방법을 구현합니다.

DINOv2는 레이블 없이 1억 4천만 개의 이미지를 사용하여 훈련되었습니다. DINOv2에 의해 생성된 임베딩은 분류, 이미지 검색, 세분화 및 깊이 추정에 사용할 수 있습니다. 그렇긴 하지만, Meta Research는 세분화 및 깊이 추정을 위한 헤드를 공개하지 않았습니다.

이 가이드에서는 DINOv2의 임베딩을 사용하여 이미지 분류기(image classification)를 빌드할 것입니다. 이를 위해 당사는 다음을 수행합니다.

1. 이미지 폴더 불러오기
2. 각 이미지에 대한 임베딩 계산
3. 모든 임베딩을 파일 및 벡터 저장소에 저장합니다.
4. SVM 분류기를 훈련시켜 영상을 분류하기

이 프로젝트에서는 [MIT Indoor Scene Recognition](https://universe.roboflow.com/popular-benchmarks/mit-indoor-scene-recognition/) 데이터 세트를 사용할 예정이지만, 가지고 있는 레이블이 지정된 모든 분류 데이터 세트를 사용할 수 있습니다.

이 노트북이 끝날 때쯤이면 데이터 세트에 대해 훈련된 분류자를 갖게 될 것입니다.

## 1. Import Packages

In [1]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
import os
import cv2
import json
import glob
from tqdm.notebook import tqdm

## 2. Load Data 데이터 불러오기

이 가이드에서는 Roboflow Universe에서 호스팅되는 MIT Indoor Scene Recognition 데이터 세트로 작업할 것입니다. 이 데이터 세트를 다운로드하려면 무료 Roboflow 계정이 필요합니다.

데이터 세트를 다운로드하고 훈련 데이터 세트의 각 이미지를 연결된 레이블에 매핑하는 사전을 만들어 보겠습니다.

In [2]:
!pip install roboflow supervision -q

In [ ]:
import roboflow
import supervision as sv

roboflow.login()

rf = roboflow.Roboflow()

project = rf.workspace("popular-benchmarks").project("mit-indoor-scene-recognition")
dataset = project.version(5).download("folder")

In [4]:
cwd = os.getcwd()

# train
ROOT_DIR = os.path.join(cwd, "MIT-Indoor-Scene-Recognition-5/train")

labels = {}

for folder in os.listdir(ROOT_DIR):
    for file in os.listdir(os.path.join(ROOT_DIR, folder)):
        if file.endswith(".jpg"):
            full_name = os.path.join(ROOT_DIR, folder, file)
            labels[full_name] = folder

files = labels.keys()

## 3. 모델 불러오기 및 계산하기 임베딩(Embeddings)

분류기를 훈련시키려면 다음이 필요합니다.

1. 데이터 세트의 각 이미지와 연결된 임베딩
2. 각 이미지와 연결된 레이블입니다

임베딩을 계산하기 위해 DINOv2를 사용합니다. 아래에서는 가장 작은 DINOv2 가중치를 로드하고 지정된 목록의 모든 이미지에 대해 임베딩을 로드하고 계산하는 함수를 정의합니다.

In [ ]:
# https://github.com/facebookresearch/dinov2#pretrained-backbones-via-pytorch-hub

dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

print(f"Device: {device}")

dinov2_vits14.to(device)

transform_image = T.Compose([
    T.ToTensor(), 
    T.Resize(364), 
    T.CenterCrop(364), 
    T.Normalize([0.5], [0.5])
])

In [ ]:
dinov2_vits14

In [7]:
def load_image(img: str) -> torch.Tensor:
    img = Image.open(img)
    
    t_img = transform_image(img)[:3].unsqueeze(0)
    
    return t_img

def compute_embedding(files: list, output_name="all_embeddings.json") -> dict:
    all_embeddings = {}
    
    with torch.no_grad():
        for i, file in enumerate(tqdm(files)):
            img = load_image(file).to(device)
            
            embeddings = dinov2_vits14(img)
            
            all_embeddings[file] = np.array(embeddings[0].cpu().numpy()).reshape(1, -1).tolist()
            
    with open(output_name, "w") as f:
        f.write(json.dumps(all_embeddings))
        
    return all_embeddings
            

## 4. 컴퓨트 임베딩(Compute Embeddings)

아래 코드는 데이터 세트의 모든 이미지에 대한 임베딩을 계산합니다. 이 단계는 MIT Indoor Scene Recognition 데이터 세트에 몇 분 정도 걸립니다. 훈련 세트에는 DINOv2를 통과해야 하는 10,000개 이상의 이미지가 있습니다.

In [ ]:
%%time
embeddings = compute_embedding(files)

## 5. 분류 모델 훈련시키기 (Train a Classification Model)

우리가 계산한 임베딩은 분류 모델에서 입력으로 사용할 수 있습니다. 이 가이드에서는 선형 분류 모델인 SVM을 사용합니다.

아래에서는 우리가 계산한 모든 임베딩과 관련 레이블의 목록을 만듭니다. 그런 다음 해당 목록을 사용하여 모델을 맞춥니다.

In [ ]:
np.array(list(embeddings.values())[0]).shape

In [ ]:
from sklearn import svm

clf = svm.SVC(gamma='scale')

y = [labels[file] for file in files]

# np.array(list(embeddings.values())[0]).shape (1, 384)
embedding_list = list(embeddings.values())

clf.fit(np.array(embedding_list).reshape(-1, 1024), y)

## 6. 분류 모델 테스트

In [ ]:
input_file = "MIT-Indoor-Scene-Recognition-5/test/elevator/elevator_google_0053_jpg.rf.41487c3b9c1690a5de26ee0218452627.jpg"

new_image = load_image(input_file)

%matplotlib inline
sv.plot_image(image=cv2.imread(input_file), size=(8, 8))

In [ ]:
with torch.no_grad():
    embedding = dinov2_vits14(new_image.to(device))

    prediction = clf.predict(np.array(embedding[0].cpu()).reshape(1, -1))

    print()
    print("Predicted class: " + prediction[0])

## 7. Evaluation 평가

In [14]:
def load_evaluation_data(root_dir):
    labels = {}
    for split in ['valid', 'test']:
        split_dir = os.path.join(root_dir, split)
        for folder in os.listdir(split_dir):
            for file in os.listdir(os.path.join(split_dir, folder)):
                if file.endswith(".jpg"):
                    full_name = os.path.join(split_dir, folder, file)
                    labels[full_name] = folder
    return labels

In [15]:
eval_root_dir = "MIT-Indoor-Scene-Recognition-5"
eval_labels = load_evaluation_data(eval_root_dir)
eval_files = list(eval_labels.keys())

In [ ]:
print("Computing embeddings for evaluation data...")
eval_embeddings = compute_embedding(eval_files, output_name="eval_embeddings.json")

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_true = []
y_pred = []

with torch.no_grad():
    for file in tqdm(eval_files):
        true_label = eval_labels[file]
        embedding = eval_embeddings[file]
        prediction = clf.predict(np.array(embedding).reshape(1, -1))
        
        y_true.append(true_label)
        y_pred.append(prediction[0])

# 평가 메트릭 계산
accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred)

print(f"\nAccuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(report)

In [ ]:
# vits14 -> Accuracy: 0.4957, f1: 0.5